In [1]:
using JLD,JLD2,Knet
using Base.Iterators: flatten
using Random, Distributions
#Random.seed!(8)

include("types.jl")
include("pre_processing.jl")
include("encoder_utils.jl")
include("decoder_utils.jl")

d = JLD.load("english_chmodel3.jld2")
d["new_d2"]["unkchar"] = Char(0x11)
d["new_d2"]["sowchar"] = Char(0x12)
d["new_d2"]["eowchar"] = Char(0x13)

v = create_vocab(d["new_d2"])
corpus = load_conllu("tr_imst-ud-train.conllu",v)
wmodel = makewmodel(d["new_d2"])

ev = extend_vocab!(v, corpus)

fs = createcolvecs(corpus,ev)

fillvecs!(wmodel, corpus, v, fs)

fillcavec!(corpus)

┌ Warning: `getval` is deprecated, use `value` instead.
│   caller = #lmloss#18(::Array{Float64,1}, ::Function, ::Array{KnetArray{Float32,2},1}, ::Array{Array{Int64,1},1}, ::Array{Array{Float32,1},1}, ::Array{Any,1}, ::Array{Any,1}) at encoder_utils.jl:63
└ @ Main /home/ec2-user/GBDP/encoder_utils.jl:63


In [2]:
s = corpus[3]

Karşısında , pantolonu dizlerine dek ıslak , önlük torbası ham eriklerle dolu İbrahim dikiliyordu . 

In [3]:
relmatrix = createrelmatrix(s)

1900×225 KnetArray{Float32,2}:
 -1.31528e-36  -1.31528e-36  -1.31528e-36  …   0.764993      0.764993   
 -2.65921e-12  -2.65921e-12  -2.65921e-12      9.42393e-10   9.42393e-10
  2.78679e-7    2.78679e-7    2.78679e-7       0.00138876    0.00138876 
  1.6633e-17    1.6633e-17    1.6633e-17      -4.5141e-10   -4.5141e-10 
 -1.21371e-6   -1.21371e-6   -1.21371e-6       1.47003e-6    1.47003e-6 
 -2.97783e-13  -2.97783e-13  -2.97783e-13  …   0.761234      0.761234   
  0.995163      0.995163      0.995163         2.28465e-19   2.28465e-19
  0.939951      0.939951      0.939951        -0.000163521  -0.000163521
 -2.12206e-12  -2.12206e-12  -2.12206e-12      0.952176      0.952176   
  2.57513e-8    2.57513e-8    2.57513e-8      -6.969e-9     -6.969e-9   
  1.13773e-13   1.13773e-13   1.13773e-13  …  -0.761648     -0.761648   
  0.999882      0.999882      0.999882        -0.00152425   -0.00152425 
 -3.51089e-9   -3.51089e-9   -3.51089e-9       8.32032e-7    8.32032e-7 
  ⋮                 

In [4]:
# Let's define a chain of layers
struct Chain
    layers
    Chain(layers...) = new(layers)
end
(c::Chain)(x) = (for l in c.layers; x = l(x); end; x)
(c::Chain)(x,y) = mynll(reshape(c(x), (length(y), length(y))),y)
#(c::Chain)(d::Data) = mean(c(x,y) for (x,y) in d)

In [5]:
struct Layer1; w; b; f; end
Layer1(i::Int,o::Int,f=relu) = Layer1(param(o,i),param0(o),f)
(l::Layer1)(x) = l.f.(l.w * x .+ l.b)

In [6]:
datasize = length(corpus[1].cavec[1]) * 2

1900

In [7]:
mlp_smodel = Chain(Layer1(datasize, 100), Layer1(100,1,identity)) # smodel = score model

Chain((Layer1(P(KnetArray{Float32,2}(100,1900)), P(KnetArray{Float32,1}(100)), Knet.relu), Layer1(P(KnetArray{Float32,2}(1,100)), P(KnetArray{Float32,1}(1)), identity)))

In [8]:
mlp_smodel(relmatrix)

1×225 KnetArray{Float32,2}:
 0.174677  0.123074  0.017305  -0.0140279  …  -0.130305  -0.259647  -0.107759

In [9]:
mlp_smodel(relmatrix, s.head)

2.7585359f0

In [14]:
epoch = 20
for e in 1:epoch
   for s in corpus #b
        if length(s)>2
            println(s.word)
            rm = createrelmatrix(s)
            println(size(rm))
            loss = sgdupdate!(mlp_smodel, (rm, s.head))
            println(loss) 
        end
    end 
    #sleep(10)
end

["Sanal", "parçacıklar", "sa", "bunların", "hiçbirini", "yapamazlar", "."]
(1900, 49)
1.920442
["Ona", "her", "şeyimi", "verdim", "."]
(1900, 25)
1.5888572
["Karşısında", ",", "pantolonu", "dizlerine", "dek", "ıslak", ",", "önlük", "torbası", "ham", "eriklerle", "dolu", "İbrahim", "dikiliyordu", "."]
(1900, 225)
2.7145953
["Nereden", "biliyorsunuz", "."]
(1900, 9)
1.0718544
["Aradığımı", "Buldum", "Sandım", "."]
(1900, 16)
1.3798177
["Kahveleri", "bende", "içelim", "."]
(1900, 16)
1.3297933
["Ne", "diyeceğimi", "bilemedim", "."]
(1900, 16)
1.3229953
["Süpürgen", "nerede", "."]
(1900, 9)
0.9110121
["Bir", "taksi", "bulduk", "."]
(1900, 16)
1.28089
["Mebrure", "birden", "heyecanlanmıştı", "."]
(1900, 16)
1.2608688
["\"", "Burada", "ki", "üst", "düzey", "görüşmelerimizde", "turizm", "için", "ellerinden", "geleni", "yapacaklarını", "söylediler", "."]
(1900, 169)
2.5120091
["Gel", "bak", ",", "sana", "evi", "göstereyim", "."]
(1900, 49)
1.8525481
["Kendileriyle", "görüştüğümden", "edindiğim

1.9280095
["Raporları", "detay", "lı", "incelemek", "için", "se", "zamanla", "yarışacaklardı", "..."]
(1900, 81)
2.026895
["Almanca", "ve", "İngilizce'yi", "çok", "iyi", "konuşan", "Jouni", ",", "sık", "sık", "yurtdışı", "seferlerine", "çıkıyordu", "."]
(1900, 196)
2.381066
["70'lere", "kadar", "gidebildim", "."]
(1900, 16)
1.1489308
["Çok", "isabet", "buyurdunuz", "."]
(1900, 16)
1.1780623
["İddia", "lı", "oldukça", "değer", "kazandıklarını", "düşünüyorlar", "."]
(1900, 49)
1.6568706
["Kız", "sabah", "olunca", "rüyasını", "hatırlamış", "."]
(1900, 36)
1.291201
["Mebrure", "yanıbaşımda", "titriyordu", "."]
(1900, 16)
0.8841626
["Korku", "dolu", "ydum", "."]
(1900, 16)
1.247016
["Ne", "iş", "yaptığımızı", "falan", "filan", "."]
(1900, 36)
1.691148
["Bir", "süre", "kapısında", "bir", "köpek", "gibi", "süründüm", "."]
(1900, 64)
1.6799031
["Ortada", "böyle", "bir", "ihtimal", "yok", "ken", "sen", "kendi", "kendine", "kuruntular", "yaratıyorsun", "."]
(1900, 144)
2.271405
["Şimdi", "birini

(1900, 256)
2.7097602
["Oysa", ",", "bunları", "kaçırmaya", "hiç", "niyeti", "yok", "tu", "."]
(1900, 81)
2.0089934
["Galiba", "haklı", "."]
(1900, 9)
0.18602091
["Bize", "uygun", "düşüyor", "."]
(1900, 16)
1.1266373
["Birkaç", "adım", "atıp", ",", "kumsalı", "okşayan", "yumuşak", "dalgaların", "çıplak", "ayaklarına", "değmesini", "bekledi", "."]
(1900, 169)
2.227353
["Konduramadım", ",", "teşhirci", "değil", "."]
(1900, 25)
1.5244919
["Tahılcı", "Anadolu", "insanı", ",", "maydanozu", "azaltıp", "bulguru", "çoğaltmış", "."]
(1900, 81)
1.7036009
["Bir", "mikrofon", ",", "ağzının", "tam", "karşısında", ",", "bir", "mikrofon", "da", "Recep'in", "darbukasının", "önünde", "dururdu", "."]
(1900, 225)
2.0768487
["Bence", "yeterince", "değil", ",", "hiç", "araştırmadan", "haber", "yapılmış", ";", "okurlar", "yanıltılmıştır", "."]
(1900, 144)
2.077351
["Fakat", "samimiyeti", "bozduğu", "için", "konuşmayı", "böyle", "resmi", "ağıza", "dönüştüren", "kişi", ",", "kendiliğinden", "soğuk", "kaçtığın

(1900, 676)
2.936682
["Stas", ",", "evin", "dışında", "ki", "yine", "kütüklerden", "inşa", "edilmiş", "depoda", "yeni", "avlanmış", "bir", "yaban", "koyununun", "derisini", "ve", "boynuzunu", "gösteriyor", "misafire", "."]
(1900, 441)
2.7950697
["Ne", "okulu", "oğlum", ",", "daha", "erken", "."]
(1900, 49)
1.628042
["Şimdi", "varırız", "oraya", ",", "dedi", "Kerem", "."]
(1900, 49)
1.585943
["Sonunda", "da", "hem", "etrafında", "kileri", "mutlu", "ediyor", "hem", "de", "kendisi", "mutlu", "oluyor", "."]
(1900, 169)
2.4367914
["Bilimde", "bu", "tür", "mucizevi", "elatmalara", "yer", "olmadığı", "gibi", ",", "varlığı", ",", "Tanrı'ya", "inananlar", "için", "tehlike", "yaratır", ":", "diferansiyel", "eşitliklerinizde", "ki", "matematiksel", "tekillikleri", "daha", "iyi", "bir", "modelle", "ortadan", "kaldırdığınızda", "Tanrınız", "da", "tekillikle", "birlikte", "ortadan", "kalkar", "."]
(1900, 1225)
3.44297
["Hadi", "hadi", "söyletme", "beni", ".."]
(1900, 25)
0.9884248
["Şaka", "bir", "y

(1900, 529)
2.9410007
["Bir", "köpeğin", "bütün", "kokuları", "birbirinden", "ayırdedip", "insanların", "duygularını", "kokularından", "anlamaları", "gibi", "kadınların", "sesini", "tanırdım", ",", "her", "tonunu", "bilirdim", ",", "yalnızca", "kulaklarımla", "değil", "bütün", "vücudumla", "duyardım", "onların", "sesini", "."]
(1900, 784)
3.026791
["Kara", "sevda", "..."]
(1900, 9)
1.3505754
["Anlarsın", "ya", "..."]
(1900, 9)
0.54266787
["Kızaklarda", "kullanılan", "tahta", ",", "deri", "ve", "boynuzların", "dışında", "tek", "tük", "halatlar", ",", "sicimler", "de", "dış", "dünyadan", "geliyor", "bu", "dağlara", "."]
(1900, 400)
2.6374
["Bir", "an", "önce", "dönmeliyim", "oraya", "."]
(1900, 36)
1.4859515
["İyi", "ki", "dedem", "var", "da", "gündüzleri", "onunla", "konuşabiliyorum", "."]
(1900, 81)
2.1322935
["Eşi", "geri", "döndükten", "sonra", "da", "bir", "süre", "İstanbul'da", "kalmayı", "planlayan", "genç", "kadın", ",", "aniden", "fikir", "değiştirip", "dönmeye", "karar", "verdi

...

(1900, 784)
3.087335
["Viski", "bardağını", "aldı", ",", "dikip", "bitirdi", "."]
(1900, 49)
0.94783986
["Ağabeyim", "hep", "CHP'den", "yana", "."]
(1900, 25)
1.2505833
["Bu", "ülkenin", "önde", "gelen", "gazetelerinin", "özel", "yayın", "ilkeleri", "arasında", "çok", "azının", "konuya", "yer", "ayırdığı", "anlaşılıyor", "."]
(1900, 256)
2.3736527
["Bu", "gece", "ulaştırmam", "gerek", "..."]
(1900, 25)
0.90611994
["Oda", "hizmetçisi", "ydi", "herhalde", "."]
(1900, 25)
0.63250625
["İnce", ",", "uzun", "parmak", "lı", "elleri", "titriyordu", "."]
(1900, 64)
0.94604576
["Bu", "hükümet", "neden", "dağıldı", "."]
(1900, 25)
0.430584
["Zamanı", "geldiğinde", "Saffet", "ile", "Recep", "sahnede", "yerlerini", "aldılar", "."]
(1900, 81)
1.2325711
["Omzunun", "üstünden", "fotoğrafı", "görebiliyordum", "."]
(1900, 25)
0.64499784
["Taşhan'ın", "duvarına", "yaslandı", "."]
(1900, 16)
0.32147443
["Evi", "ev", "yapan", ",", "duvarlarına", "asılan", "resim", "lerdir", ",", "tablo", "lardır", ",", "ku

(1900, 324)
2.320841
["Göstergelerde", "bir", "anormallik", "oldu", "."]
(1900, 25)
0.531743
["yedi", "yıl", "baba", "olma", "hasretiyle", "yanan", "Murat", "Öğretmen", ",", "ikibuçuk", "ay", "önce", "dünyaya", "gelen", "kızı", "İrem", "ve", "eşi", "Eser'i", "kazada", "kaybetti", "."]
(1900, 484)
2.6547294
["Eşi", "ve", "kızı", "öldü", "ama", "o", "bilmiyor", "."]
(1900, 64)
1.1552973
["Uçaktan", "indiğimde", ",", "Üstünde", "kiler", "olmaz", "."]
(1900, 49)
1.0979791
["-", "Okuyorum", "."]
(1900, 9)
0.35850897
["Kuantum", "kuramının", "öngörüsü", "olan", "sanal", "parçacıklar", "metafizik", "kavram", "lardır", "(", ")", "."]
(1900, 144)
1.5113808
["Bu", "rahat", "davranış", "lı", "New", "York", "lu", "kız", ",", "tedirginlikleri", "büyütmeye", "elverişli", "içimi", "sakinleştirmişti", "."]
(1900, 225)
2.0267017
["Bilim", "tarihinde", ",", "çok", "uzun", "ve", "zahmet", "li", "bir", "kolektif", "sürecin", "ürünü", "olan", "mantığı", "saymazsak", ",", "iki", "önem", "li", "teorik", "atı

["Kemal'in", "sözlerini", "tekrarlayarak", ",", "sevgilinin", "uyurken", "ki", "beş", "ayrı", "halini", "yirmisekizer", ",", "Hülya'nın", "gözkapaklarına", "çöküp", "toplanan", "buğunun", "renklerini", "otuzbeşer", "defa", "gözlerimin", "önünden", "geçirdikten", "sonra", ",", "uyuyamayacağımı", "anlayıp", "doğruldum", "."]


Excessive output truncated after 524288 bytes.

(1900, 841)
2.6452446
["Buz", "kesiyorum", "gitgide", ",", "aldırdığı", "yok", "."]
(1900, 49)
2.0935245
["Kolayca", "eşelenen", "kuru

In [15]:
# ********* silmeeee
accuracy(reshape(mlp_smodel(createrelmatrix(s)), (length(s),length(s))), s.head)

0.8571428571428571

In [10]:
ypred = reshape(mlp_smodel(relmatrix),(length(s),length(s)))

15×15 KnetArray{Float32,2}:
  0.174677    0.187226     0.0423454   0.076674    …  -0.00164204   0.0736273
  0.123074    0.110824     0.0853871   0.0799069       0.0724677   -0.096365 
  0.017305    0.0228608   -0.0793054  -0.00858774     -0.102365    -0.267392 
 -0.0140279  -0.0161092   -0.0832986  -0.0406346      -0.156786    -0.232704 
  0.0250729  -0.0372137   -0.0747297  -0.0581685      -0.0998087   -0.197356 
  0.126875    0.16896      0.0697326   0.0765567   …  -0.0260273   -0.0754644
  0.140138    0.132727     0.0714913   0.0917166       0.0441149   -0.128321 
  0.0527001  -0.0225152   -0.0818856  -0.0283098      -0.080874    -0.203259 
 -0.025858   -0.0891147   -0.133805   -0.114877       -0.200823    -0.293464 
  0.0468694   0.0695372   -0.0168438  -0.0614115      -0.100664    -0.145261 
 -0.0343694  -0.0884452   -0.131849   -0.116347    …  -0.207348    -0.306447 
  0.103555    0.109069     0.0587676   0.0418056       0.0103491   -0.0673901
  0.0374773   0.00771314  -0.053276 

In [11]:
ypredroott = ypredroot(ypred,s.head)

16×16 KnetArray{Float32,2}:
 0.0   0.0         0.0          0.0        …   1.0          0.0      
 0.0   0.174677    0.187226     0.0423454     -0.00164204   0.0736273
 0.0   0.123074    0.110824     0.0853871      0.0724677   -0.096365 
 0.0   0.017305    0.0228608   -0.0793054     -0.102365    -0.267392 
 0.0  -0.0140279  -0.0161092   -0.0832986     -0.156786    -0.232704 
 0.0   0.0250729  -0.0372137   -0.0747297  …  -0.0998087   -0.197356 
 0.0   0.126875    0.16896      0.0697326     -0.0260273   -0.0754644
 0.0   0.140138    0.132727     0.0714913      0.0441149   -0.128321 
 0.0   0.0527001  -0.0225152   -0.0818856     -0.080874    -0.203259 
 0.0  -0.025858   -0.0891147   -0.133805      -0.200823    -0.293464 
 0.0   0.0468694   0.0695372   -0.0168438  …  -0.100664    -0.145261 
 0.0  -0.0343694  -0.0884452   -0.131849      -0.207348    -0.306447 
 0.0   0.103555    0.109069     0.0587676      0.0103491   -0.0673901
 0.0   0.0374773   0.00771314  -0.053276      -0.15444     -0.

In [12]:
eisnerheads = eisner(ypredroott)

15-element Array{Int64,1}:
 12
  1
  2
  2
  6
  2
  2
  2
  2
  2
  2
  0
 12
  0
  0

In [13]:
maxheads = findmaxscores(ypredroott)
maxheads = map(x->x-1,maxheads)[2:length(maxheads)]

15-element Array{Int64,1}:
  1
  1
  2
  7
  6
  2
  2
  1
  2
 12
  2
  1
  1
  0
  1

In [15]:
# Let's define a chain of layers
struct Chain2
    layers
    Chain2(layers...) = new(layers)
end
(c::Chain2)(x) = (for l in c.layers; x = l(x); end; x)
(c::Chain2)(x,y) = mynll(reshape(c(x), (size(fs.deprels,2), length(y))),y)
#(c::Chain)(d::Data) = mean(c(x,y) for (x,y) in d)

In [17]:
decidermodel = Chain2(Layer1(length(corpus[1].cavec[1]) * 3 + tagrepsize * 100, 100), Layer1(100,1,identity)) # smodel = score model

Chain2((Layer1(P(KnetArray{Float32,2}(100,6050)), P(KnetArray{Float32,1}(100)), Knet.relu), Layer1(P(KnetArray{Float32,2}(1,100)), P(KnetArray{Float32,1}(1)), identity)))

In [18]:
deciderinput = createdeciderinput(s, maxheads, eisnerheads, fs.deprels)

6050×555 KnetArray{Float32,2}:
 -1.31528e-36  -1.31528e-36  -1.31528e-36  …   0.764993      0.764993   
 -2.65921e-12  -2.65921e-12  -2.65921e-12      9.42393e-10   9.42393e-10
  2.78679e-7    2.78679e-7    2.78679e-7       0.00138876    0.00138876 
  1.6633e-17    1.6633e-17    1.6633e-17      -4.5141e-10   -4.5141e-10 
 -1.21371e-6   -1.21371e-6   -1.21371e-6       1.47003e-6    1.47003e-6 
 -2.97783e-13  -2.97783e-13  -2.97783e-13  …   0.761234      0.761234   
  0.995163      0.995163      0.995163         2.28465e-19   2.28465e-19
  0.939951      0.939951      0.939951        -0.000163521  -0.000163521
 -2.12206e-12  -2.12206e-12  -2.12206e-12      0.952176      0.952176   
  2.57513e-8    2.57513e-8    2.57513e-8      -6.969e-9     -6.969e-9   
  1.13773e-13   1.13773e-13   1.13773e-13  …  -0.761648     -0.761648   
  0.999882      0.999882      0.999882        -0.00152425   -0.00152425 
 -3.51089e-9   -3.51089e-9   -3.51089e-9       8.32032e-7    8.32032e-7 
  ⋮                 

In [27]:
reshape(decidermodel(deciderinput), (37,7))

37×7 KnetArray{Float32,2}:
 -0.201004   -0.344739   -0.226494   …  -0.290059   -0.341806   -0.227712  
 -0.106649   -0.341239   -0.185648      -0.313594   -0.336067   -0.238623  
 -0.397473   -0.505274   -0.285074      -0.433643   -0.508636   -0.302762  
 -0.0756993  -0.327271   -0.105408      -0.23659    -0.362053   -0.170701  
 -0.03819    -0.197351    0.011367      -0.0822473  -0.25714    -0.0260708 
  0.655412    0.508522    0.77031    …   0.540796    0.465574    0.702791  
 -0.51413    -0.597948   -0.40565       -0.569233   -0.620896   -0.420367  
 -0.0604682  -0.139374    0.0404108     -0.0728283  -0.152461    0.01077   
 -0.807643   -0.802237   -0.693868      -0.793235   -0.901488   -0.684602  
 -0.43468    -0.587344   -0.418109      -0.554373   -0.579001   -0.427908  
 -0.252921   -0.392843   -0.237458   …  -0.337224   -0.485605   -0.263926  
  0.10491     0.0375227   0.121149       0.122779    0.0431997   0.241866  
 -0.509439   -0.623856   -0.497248      -0.660128   -0.647298

In [19]:
decidermodel(deciderinput, s.deprel)

3.5026867f0

In [20]:
#for e in 1:epoch
   for s in corpus #b
        if length(s)>2
            println(s.word)
            ypred = ypredroot(reshape(mlp_smodel(createrelmatrix(s)),(length(s),length(s))),s.head)
            maxheads = findmaxscores(ypred)
            maxheads = map(x->x-1,maxheads)[2:length(maxheads)] #refactor so that root excluded
            eisnerheads = eisner(ypred)
            di = createdeciderinput(s, maxheads, eisnerheads, fs.deprels)
            println(size(di))
            loss = sgdupdate!(decidermodel, (di, s.deprel))
            println(loss) 
        end
    end 
    #sleep(10)
#end

["Sanal", "parçacıklar", "sa", "bunların", "hiçbirini", "yapamazlar", "."]
(6050, 259)
3.5537083
["Ona", "her", "şeyimi", "verdim", "."]
(6050, 185)
3.3570755
["Karşısında", ",", "pantolonu", "dizlerine", "dek", "ıslak", ",", "önlük", "torbası", "ham", "eriklerle", "dolu", "İbrahim", "dikiliyordu", "."]
(6050, 555)
3.0856576
["Nereden", "biliyorsunuz", "."]
(6050, 111)
2.3103666
["Aradığımı", "Buldum", "Sandım", "."]
(6050, 148)
2.5577989
["Kahveleri", "bende", "içelim", "."]
(6050, 148)
1.840455
["Ne", "diyeceğimi", "bilemedim", "."]
(6050, 148)
1.5047472
["Süpürgen", "nerede", "."]
(6050, 111)
2.790327
["Bir", "taksi", "bulduk", "."]
(6050, 148)
2.4492795
["Mebrure", "birden", "heyecanlanmıştı", "."]
(6050, 148)
2.689541
["\"", "Burada", "ki", "üst", "düzey", "görüşmelerimizde", "turizm", "için", "ellerinden", "geleni", "yapacaklarını", "söylediler", "."]
(6050, 481)
3.3970163
["Gel", "bak", ",", "sana", "evi", "göstereyim", "."]
(6050, 259)
2.7677298
["Kendileriyle", "görüştüğümden"

(6050, 148)
2.1665597
["Ütüden", "anlamam", "."]
(6050, 111)
1.5819343
["Yanakları", "hafifçe", "pembeleşmişti", "."]
(6050, 148)
2.1270318
["Siyah", "rugan", "."]
(6050, 111)
1.896147
["Bu", ",", "tersinir", "bir", "süreç", "tir", "."]
(6050, 259)
2.8359654
["Evet", ",", "ilk", "kocası", "."]
(6050, 185)
1.8079274
["Hele", "o", "zamanlar", "moda", "olan", "sizler", "sözcüğüne", "ben", "hala", "şaşarım", "."]
(6050, 407)
2.9922764
["Yattığı", "yerde", ",", "kumların", "üzerinde", "gerinebildiği", "kadar", "gerindi", "."]
(6050, 333)
2.777553
["Raporları", "detay", "lı", "incelemek", "için", "se", "zamanla", "yarışacaklardı", "..."]
(6050, 333)
2.6830668
["Almanca", "ve", "İngilizce'yi", "çok", "iyi", "konuşan", "Jouni", ",", "sık", "sık", "yurtdışı", "seferlerine", "çıkıyordu", "."]
(6050, 518)
2.6401777
["70'lere", "kadar", "gidebildim", "."]
(6050, 148)
2.5158632
["Çok", "isabet", "buyurdunuz", "."]
(6050, 148)
2.1779892
["İddia", "lı", "oldukça", "değer", "kazandıklarını", "düşünüyo

(6050, 185)
1.7981366
["Sana", "açmayacağıma", "söz", "vermiştim", "..."]
(6050, 185)
2.4560316
["Pantolon", "imalatıyla", "ilgili", "bütün", "tasarılar", "gerçek", "oldu", "."]
(6050, 296)
2.0848234
["Evlilik", "aşkı", "desek", "!", ".."]
(6050, 185)
1.3353354
["Park", "kurşun", "sesleri", "ile", "yankılanmış", "."]
(6050, 222)
2.134825
["Eski", "sevgilisi", "kapının", "dışında", "bekliyor", "."]
(6050, 222)
2.088687
["Her", "yanım", "titriyor", ",", "ter", "içinde", "kaldım", "."]
(6050, 296)
2.339211
["Tek", "hesabın", "kapsamı", "genişliyor", "."]
(6050, 185)
1.5271553
["Kız", "içeri", "girmedi", "."]
(6050, 148)
1.5587955
["Ben", "tarafsız", "."]
(6050, 111)
0.865275
["Göklere", "çıkarıyordun", "."]
(6050, 111)
1.4927486
["Halbuki", "şu", "anda", "işyerlerinin", "sağladığı", "finansmanla", "bu", "iş", "yürüyor", "\"", "dedi", "."]
(6050, 444)
2.8747299
["Ve", "fakat", "bu", "yeni", "gerçek", ",", "Kadıköy", "vapurunun", "karşıdan", "karşıya", "geçiş", "süresini", "hiçbir", "şekild

(6050, 518)
2.3333635
["Dönmüyor", "da", "zaten", "..."]
(6050, 148)
2.0583265
["Yolu", "yarılamıştık", "neredeyse", "."]
(6050, 148)
1.7672439
["Erkek", "bir", "makine", "bu", "."]
(6050, 185)
2.2648685
["Evenlerin", "tarihte", "dokuz", "devlet", "kurduğunu", "ve", "bunlardan", "dokuz.", "yüzyılda", "kurulan", "Bahai", "ile", "oniki.", "yüzyılda", "kurulan", "Sin'in", "(", "Altın", ")", "imparatorluk", "olduğunu", "da", "belirtti", "."]
(6050, 888)
2.7541063
["Her", "gün", "yüz", "bin", "mi", "verecekler", "sana", "."]
(6050, 296)
3.1439085
["Hadi", "bakalım", "düşelim", "yola", ".", "dedi", "babam", "."]
(6050, 296)
2.39041
["Siz", "gene", "de", "demeyin", "."]
(6050, 185)
1.8162874
["Mahmut", "Beyi", "aramıştım", "."]
(6050, 148)
2.2397232
["Biraz", "da", "elleri", "sıkı", "dır", "."]
(6050, 222)
2.1028786
["J", ",", "ziyareti", "sırasında", "Türkiye", "ekonomisinin", "genel", "durumu", "üzerinde", "de", "duracaklarını", "ifade", "etti", "."]
(6050, 518)
2.2399757
["Beni", "sınava",

(6050, 740)
2.3481214
["Bütün", "bunlara", "rağmen", ",", "Sav", "familyası", "imajını", "zedelemeyerek", "aynı", "okulları", "bitirdim", "ben", "de", "."]
(6050, 518)
2.329747
["'", "seçim", "döneminde", "kendisini", "Çankaya", "Köşkü'ne", "çıkaracağı", "yolunda", "Milli", "Görüş", "tabanına", "propaganda", "yaptığını", "öne", "süren", "kapatılan", "RP'nin", "Genel", "Başkanı", "Necmettin", "Erbakan'a", "'", "milletvekilleri", "sert", "tepki", "gösterdi", "."]
(6050, 999)
2.614871
["Ağzının", "kenarında", "var", "mıydı", "bu", "derin", "çizgi", "?"]
(6050, 296)
2.6737702
["Patlıcan", ",", "deyimlerimize", "de", "girmiştir", "."]
(6050, 222)
1.6541685
["Bu", "da", "bölgede", "ki", "ticarete", "katkıda", "bulunacak", "."]
(6050, 296)
1.9960176
["Durun", ",", "oturun", "biraz", "."]
(6050, 185)
1.498262
["Kadın", "uyuduysa", "okur", ",", "dedi", "adam", "."]
(6050, 259)
1.6967664
["Bilimsel", "Devrim", "boyunca", "alanını", "din", "aleyhine", "gittikçe", "genişleterek", ",", "dinin", "es

...

(6050, 481)
1.7446437
["Kapıya", "yürüdüm", ",", "açtım", "."]
(6050, 185)
1.1838745
["Altı", "saat", "lik", "bir", "koşu", ",", "kervanı", "Seben", "kasabasına", "en", "yakında", "kurulu", "çoban", "barınağına", "getiriyor", "."]
(6050, 592)
1.7827328
["Sezer", "veto", "edebilir", ",", "referanduma", "getirebilir", "."]
(6050, 259)
1.5337679
["Anne", "buna", "itiraz", "edince", "adam", "belinde", "ki", "kayışı", "çıkarıp", "anne", "ile", "kızı", "öldüresiye", "dövmüş", "."]
(6050, 555)
1.8952582
["Onu", "en", "iyi", "okullarda", "okuttum", "diyerek", "kendisini", "rahatlatma", "yollarını", "arıyordu", "."]
(6050, 407)
1.9018987
["Evet", ",", "gönlüm", "ablasından", "yanay", "dı", "."]
(6050, 259)
0.9534543
["Sanki", "o", "konuştukça", "beynim", "daha", "bir", "kışkırtılıyor", ",", "o", "konuştukça", "anlattığı", "tarihlere", "denk", "düşen", "öbür", "görüntüler", "aynı", "tarihe", "paralel", "yürüyüp", "gidiyor", "."]
(6050, 814)
2.0508804
["Ne", "derdik", "."]
(6050, 111)
0.85309476


(6050, 1332)
2.0577395
["Kabloda", "güç", "birliği", "yaptılar", "."]
(6050, 185)
0.89291805
["Hayret", "içinde", "ydim", "."]
(6050, 148)
1.2259253
["Buna", "kapılalı", "beri", "bana", "seyrek", "gelir", "oldun", "."]
(6050, 296)
2.0977364
["O", "zaman", "damda", ",", "ineklerle", "yatsın", "."]
(6050, 259)
1.2338519
["Aslında", "sakızla", "teması", "yok", "."]
(6050, 185)
1.4272242
["Çünkü", ",", "sanık", "sandalyesinde", "oturan", "darbecilerden", "birisi", "kalkıp", "dinleyici", "sıralarına", "parmağını", "uzatsa", "ve", "sorsa", "."]
(6050, 555)
2.0923233
["Kadınla", "kızı", "sonunda", "aç", "kalmış", "."]
(6050, 222)
2.0594363
["Karşı", "karşıya", "yız", "."]
(6050, 148)
2.0063906
["Ben", "yeni", "yedim", "."]
(6050, 148)
1.6210276
["Ayırmak", "için", "Zerzevat", "dedim", "."]
(6050, 185)
1.5208249
["Olayın", "ikiyüzkırkyedi", "mağduru", "olduğu", "ifade", "edilen", "fezlekede", ",", "Akgündüz'ün", ",", "TCK'nın", "\"", "W", "dolandırıcılık", "\"", "suçunu", "tanımlayan", "beşyüz

1.9260722
["Patlıcan", "erdemli", "bir", "sebze", "dir", "."]
(6050, 222)
1.19261
["Mantık", "yedi", "yaşından", "itibaren", "devreye", "girer", "."]
(6050, 259)
2.1171088
["Felsefeye", "merakı", "onu", "ünlü", "yazar", ",", "çizer", "ve", "ressamların", "dünyasına", "sokuyor", "."]
(6050, 444)
1.745796
["oniki", "Aralık'a", "kadar", "çıkarılması", "beklenen", "AB", "uyum", "yasalarını", "içeren", "paketin", "çalışmaları", "dün", "gece", "tamamlandı", "."]
(6050, 555)
2.9058988
["Beyzadeler", "bizi", "ne", "sanıyorlar", "."]
(6050, 185)
1.3182477
["Ama", "şimdi", "o", "anlatmak", "istediklerimi", "nasıl", "anlatacağımı", "bilmiyorum", "."]
(6050, 333)
2.107081
["Ya", "dayak", "yerseniz", ",", "ya", "karakola", "gidersiniz", "."]
(6050, 296)
1.5192807
["Bira", ",", "o", "kadar", ",", "kaldıramıyorum", "sert", "içkileri", "."]
(6050, 333)
1.3604475
[",", "Bonham'a", "gidemem", ",", "Hülya'nın", "yanında", "kalmalıyım", ";", "ya", "da", "arkasında", ",", "cephe", "gerisinde", "..."]
(6050

(6050, 851)
2.1006386
["Daha", "da", "kötüsü", ",", "bu", "gerçeği", "bilmeme", "rağmen", ",", "her", "seferinde", "elimde", "olmadan", "hep", "aynı", "şeyi", "yapıyordum", ",", "bunun", "önüne", "geçemiyordum", "."]
(6050, 814)
1.6887398
["Avrupalılara", "da", "kabul", "ettirmişlerdir", "."]
(6050, 185)
1.1095618
["Bu", "çekinme", "siz", ",", "sıcak", "davranış", "lı", ",", "içinde", "biriktirdiği", "derinliği", "dışa", "da", "yansıtmaktan", "kaçınmayan", "sarışın", "kız", ",", "gelebileceğini", "söylemişti", "."]
(6050, 777)
1.8279064
["Telaşla", "ayağa", "fırladı", ",", "telefona", "koşarken", "kapı", "vuruldu", "."]
(6050, 333)
1.2196575
["Ardımdan", "da", "kendimi", "gördüm", ":", "kanalların", "suları", "üzerinde", ",", "kendiliğinden", "yol", "alan", "bir", "sandalda", ",", "daha", "büyük", "kanallara", "geçen", ",", "açık", "denize", "doğru", "yol", "alan", "kendimi", "."]
(6050, 999)
1.7344568
["Merak", "ediyormuş", "o", "da", "evimi", "."]
(6050, 222)
1.604043
["Doldurduğu", 

(6050, 518)
1.3107591
["çalışmak", "da", "değil", ",", "hayatla", "boğuşuyorlar", "."]
(6050, 259)
0.868981
["Kafamı", "dinleyeceğim", "\"", "dedi", "."]
(6050, 185)
0.992033
["Nasıl", "çıkabilirim", "oraya", "."]
(6050, 148)
2.177112
["Resmi", "ve", "alternatif", "tarih", "tartışmalarını", ",", "kültürün", "tozlarından", "arınması", "olarak", "görmek", "de", "en", "doğrusu", "..."]
(6050, 555)
1.8430039
["Pamuk", "Prenses", "masalının", "üvey", "babaya", "uyarlanması", "olmuş", "bence", "."]
(6050, 333)
2.0392385
["Anlatılmalı", "bir", "gün", "."]
(6050, 148)
1.2714138
["Eyvah", "!", "Mantara", "basıyoruz", "galiba", "."]
(6050, 222)
1.8920362
["Sobanın", "üstünde", "içme", "suyuna", "dönüşecek", "daha", "sonra", "bunlar", "."]
(6050, 333)
1.8282388
["Heyette", "yer", "alan", "CHP", "İzmir", "Milletvekili", "Erdal", "Karademir", ",", "ziyaret", "sırasında", ",", "üç", "ay", "önce", "üniversite", "eğitimi", "için", "Kıbrıs'a", "gönderdiği", "oğlu", "Utku", "ile", "buluştu", "."]
(6050,

In [24]:
function fillparsers!(corpus, deprels)
    for s in corpus
            ypred = mlp_smodel(createrelmatrix(s))
            ypred = ypredroot(reshape(ypred, (length(s),length(s))),s.head)
            maxheads = findmaxscores(ypred)
            maxheads = map(x->x-1,maxheads)[2:length(maxheads)] #refactor so that root excluded
            eisnerheads = eisner(ypred)
            di = createdeciderinput(s, maxheads, eisnerheads, deprels)
        s.parse = Parser(eisner(ypred), findmaxscores(reshape(decidermodel(di), (size(deprels,2),length(s)) )))
    end
end

fillparsers! (generic function with 1 method)

In [25]:
fillparsers!(corpus, fs.deprels)

In [26]:
s.parse.deprel

15-element Array{UInt8,1}:
 0x1c
 0x22
 0x1b
 0x1b
 0x04
 0x1b
 0x22
 0x1b
 0x1b
 0x1b
 0x1b
 0x0c
 0x1e
 0x01
 0x22

In [27]:
function las(corpus)
    nword = ncorr = 0
    for s in corpus
        p = s.parse
        nword += length(s)
        ncorr += sum((s.head .== p.head) .& (s.deprel .== p.deprel))
    end
    ncorr / nword
end

las (generic function with 1 method)

In [28]:
las(corpus)

0.14135766654359408

In [82]:
model_dict = Dict()

push!(model_dict, "smodel_f2" => mlp_smodel.layers[2].f)

Dict{Any,Any} with 0 entries

In [95]:
using FileIO
FileIO.save("mlpmodels.jld2", model_dict)

In [96]:
#=mmm = load("mlpmodels.jld2")

layer1 = Layer1(mmm["smodel_w1"], mmm["smodel_b1"], mmm["smodel_f1"])

layer2 = Layer1(mmm["smodel_w2"], mmm["smodel_b2"], mmm["smodel_f2"])

newmodel = Chain(layer1, layer2)=#

Dict{String,Any} with 12 entries:
  "smodel_f1"       => relu
  "smodel_b2"       => Float32[0.0]
  "smodel_w2"       => Float32[-0.137809 0.11007 … 0.0387697 -0.1218]
  "decidermodel_b2" => Float32[1.46431e-7]
  "decidermodel_f1" => relu
  "smodel_f2"       => identity
  "decidermodel_b1" => Float32[0.000686876, -0.0058242, 6.80275e-5, -0.00036977…
  "decidermodel_f2" => identity
  "smodel_b1"       => Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0…
  "decidermodel_w2" => Float32[0.0409463 -0.363427 … -0.193033 0.0415602]
  "smodel_w1"       => Float32[-0.0294824 -0.000274695 … -0.0187788 -0.00532644…
  "decidermodel_w1" => Float32[-0.0119954 -0.00659097 … -0.01209 -6.35465e-5; -…